In [58]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

#### Use splinter and chromedriver to download html source

In [3]:
# Path to chromedriver
path=!which chromedriver
executable_path = {'executable_path': path[0]}
browser = Browser('chrome', **executable_path, headless=False)

## Scrape Mars NASA News Site

In [27]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [28]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [29]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth'

In [30]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight."

#### Featured image

In [31]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

##### id is unique in html source

In [32]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [16]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)


False

In [33]:
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [34]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [35]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA17564_hires.jpg'

In [36]:
host_url = 'www.jpl.nasa.gov'
img_url_full = f'{host_url}{img_url_rel}'
img_url_full

'www.jpl.nasa.gov/spaceimages/images/largesize/PIA17564_hires.jpg'

### Pandas parsing html table

In [21]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [77]:
df.to_html(classes="table table-condensed table-striped table-hover")

'<table border="1" class="dataframe table table-condensed table-striped table-hover">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\

### Scraping Mars Hemisphere

In [63]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
host_url = 'https://astrogeology.usgs.gov'
browser.visit(url)

In [64]:
html = browser.html
hemi_soup = soup(html, 'html.parser')

#### Scrape item links

In [65]:
links = hemi_soup.find_all('a', class_='itemLink')
links[:2]

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>]

#### Extract link, thumbnail, and title

In [66]:
from collections import defaultdict
link_dict = defaultdict(dict)
for link in links:
    href = link['href']
    thumb = link.find('img')
    if thumb:
        link_dict[href]['thumb']= thumb.get('src')
    
    title = link.find('h3')
    if title:
        link_dict[href]['title'] = title.get_text()
        
mars_hemisphere = [{'href': href, 'title': link_dict[href]['title'], 'thumb': link_dict[href]['thumb']} for href in link_dict]
mars_hemisphere

[{'href': '/search/map/Mars/Viking/cerberus_enhanced',
  'title': 'Cerberus Hemisphere Enhanced',
  'thumb': '/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'},
 {'href': '/search/map/Mars/Viking/schiaparelli_enhanced',
  'title': 'Schiaparelli Hemisphere Enhanced',
  'thumb': '/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'},
 {'href': '/search/map/Mars/Viking/syrtis_major_enhanced',
  'title': 'Syrtis Major Hemisphere Enhanced',
  'thumb': '/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'},
 {'href': '/search/map/Mars/Viking/valles_marineris_enhanced',
  'title': 'Valles Marineris Hemisphere Enhanced',
  'thumb': '/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]

#### Follow link to get full image URL

In [74]:
for item in mars_hemisphere:
    url = f"{host_url}{item['href']}"
    browser.visit(url)
    html = browser.html
    img_soup = soup(html, 'html.parser')
    img_url = img_soup.find('a', string='Sample')['href']
    item['img_url'] = img_url
mars_hemisphere    

[{'href': '/search/map/Mars/Viking/cerberus_enhanced',
  'title': 'Cerberus Hemisphere Enhanced',
  'thumb': '/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'href': '/search/map/Mars/Viking/schiaparelli_enhanced',
  'title': 'Schiaparelli Hemisphere Enhanced',
  'thumb': '/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'href': '/search/map/Mars/Viking/syrtis_major_enhanced',
  'title': 'Syrtis Major Hemisphere Enhanced',
  'thumb': '/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'href': '/search/map/Mars/Viking/valles_marineris_enhanced',
  

In [75]:
browser.quit()